### Reproducible Workflow for the Undergraduate Computational Chemistry Lab  

Jacob States, Isaac Spackman, Shubham Vyas  
Colorado School of Mines

This notebook demonstrates a workflow accessible to undergraduate physical chemistry students to support the use of research software to facilitate quantum mechanical modeling. Each small molecule radical investigated in this experiment consists of a central carbon atom and 3 different heteroatoms (H, F, Cl, Br, or I). To compute meaningful data using a separate quantum mechanics modeling software (ORCA), students need to generate input guess geometries and write the input files for the program. When the quantum mechanics modeling is complete, students need to parse through long log files detailing the results of the computations and present the data in a manner that enables further analysis.  

The workflow introduces students to the capabilities of the common datascience pachage pandas, in addition to tools specific to the field of computational chemistry (rdkit and cclib).  

_Overview_  
    (1) Load experimental plan from an excel sheet  
    (2) Generate input molecular geometries from SMILES strings  
    (3) Write appropriate input files for quantum mechanics modeling software  
    (3.5) Run the quantum mechanics modeling (not included in this notebook)  
    (4) Parse the output files to extract data and write to a csv file  



In [ ]:
#############################################################
# Module Import
#############################################################
import os
import re
import pandas as pd
import cclib.io as cc
from rdkit import Chem
from rdkit.Chem import AllChem

#### (1,2) Load Experimental Plan and Write Input Geometries  

This notebook emphasizes the use of a planned experiment template excel file containing all combinations of small organic radicals with one carbon atom and all combinations of the different halogens (F, Cl, Br, I). Students can use this single input file to generate starting 3D molecular structures for their quantum mechanical simulations in a reproducible fashion. Because this step is automated, students can examine many more structures that would normally be possible in a 3 hour lab period.  
  
The input structures are generated from SMILES strings, which are a cheminformatic convention describing molecular structures in a simple typographic manner. One complexity students may encounter that is handled by the code below is that SMILES strings utilize implicit hydrogen atoms. This means that the hydrogen atoms in the molecular structures are not expressed in the strings themselves, rather they are inferred by the operating package rdkit. This can lead to challenges writing the geometries.

In [ ]:
#############################################################
# LOAD INPUT DATA
#############################################################


# define relative path to input dataframe
CX3_dataframe = ".\\dataframes\\CX3_radical_dataframe.xlsx"
# read file into pandas DataFrame object
df = pd.read_excel(CX3_dataframe)
# sorts molecules to be generated in alphabetical order
df = df.sort_values(["atomA","atomB","atomC"], ignore_index=True)


#############################################################
# GENERATE INPUT XYZ COORDINATES
#############################################################


# loop through experiment dataframe and generate xyz input coordinates
parsed_rows = []
for _, row in df.iterrows():
    # use rdkit to convert SMILES to XYZ
    smiles = "[{atomA}][C]([{atomB}])[{atomC}]".format(**row)
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol) # Adds modifications (the hydrogens are added) to the molecule file
    mol_xyz = pd.Series(Chem.MolToXYZBlock(mol)[1:].split())
    # Check to see if atomA and C coordinates were swapped during the AddHs method then swap their positions
    # This occurs due to the implicit hydrogens in the SMILES strings
    atomA, atomB, atomC = mol_xyz[0], mol_xyz[8], mol_xyz[12]
    if atomA == 'C':
        i = 0
        while i <= 3:
            mol_xyz[i], mol_xyz[i+4] = mol_xyz[i+4], mol_xyz[i]
            i+=1
    else:
        pass

    # extract data from the molecule xyz coordinate file and convert numbers to strings
    # the three heteroatoms are denoted A, B, or C
    atomA, atomB, atomC = mol_xyz[0], mol_xyz[8], mol_xyz[12]
    Cx_in, Cy_in, Cz_in = float(mol_xyz[5]), float(mol_xyz[6]), float(mol_xyz[7])
    xA_in, yA_in, zA_in = float(mol_xyz[1]), float(mol_xyz[2]), float(mol_xyz[3])
    xB_in, yB_in, zB_in = float(mol_xyz[9]), float(mol_xyz[10]), float(mol_xyz[11])
    xC_in, yC_in, zC_in = float(mol_xyz[13]), float(mol_xyz[14]), float(mol_xyz[15])

    # place data and coordinates into a dictionary then append to list
    data_dict = {"atomA" : atomA, "atomB": atomB, "atomC" : atomC, 
                 "Cx_in" : Cx_in, "Cy_in" : Cy_in, "Cz_in" : Cz_in,
                 "xA_in" : xA_in, "yA_in" : yA_in, "zA_in" : zA_in,
                 "xB_in" : xB_in, "yB_in" : yB_in, "zB_in" : zB_in,
                 "xC_in" : xC_in, "yC_in" : yC_in, "zC_in" : zC_in,}
    parsed_row = pd.Series(data = data_dict)
    parsed_rows.append(parsed_row)

# convert parsed rows to dataframe
parsed_data = pd.DataFrame(parsed_rows)


#############################################################
# WRITE DATA TO DATA FRAME AND EXCEL
#############################################################


# reload original input dataframe (or you could make a completely new one)
CX3_dataframe = ".\\dataframes\\CX3_radical_dataframe.xlsx"
CX3_dataframe_input = ".\\dataframes\\CX3_radical_dataframe_input.xlsx"
# read file into pandas DataFrame object
df = pd.read_excel(CX3_dataframe)
# sorts files before
df = df.sort_values(["atomA","atomB","atomC"], ignore_index= True)
# populates data input coordinates into original dataframe
df.update(parsed_data)

# update the dataframe excel file with new data
# note this will over-write the current file, so any formatting or plots will be lost
# you could save to a new name to avoid this issue
df.to_excel(CX3_dataframe_input, index=False)

### (3) Write Input Files for Quantum Mechanics Modeling  
  
To simplify the process of generating multiple similar input files for the quantum mechanics modeling program ORCA, we developed a template file that students can edit programmatically. By loading the guess 3D geometries for the molecules from their experimental plan, they can iterate through the list to write the data to the input template. This ensures input files are reproducible and significantly speeds up the process of preparing for quantum calculations. This example script also makes use of regular expressions (regex) which are an essential tool of any computational chemist.  

In [ ]:
#############################################################
# LOAD TEMPLATE
#############################################################

# define relative path to template file
CX3_radical_template = ".\\templates\\CX3_radical_template.txt"
# read template as a string
with open(CX3_radical_template, "r") as f:
    CX3_radical_template = f.read()
# get the set of required template keys
regex = re.compile(r'(?<!\{)\{([^{}]+)\}(?!\})') # Looks through template file for all strings inside {}
matches = set(regex.findall(CX3_radical_template))
template_keys = {item.split(":")[0] for item in matches}


#############################################################
# LOAD INPUT DATA
#############################################################

# define relative path to input dataframe
CX3_dataframe_input = ".\\dataframes\\CX3_radical_dataframe_input.xlsx"
# read file into pandas DataFrame object
df = pd.read_excel(CX3_dataframe_input)
# add molecule name column (format or change name however you like)
df["molecule"] = ["C-{atomA}-{atomB}-{atomC}_radical".format(**row) for _,row in df.iterrows()]

# check that each key required by the template has values that are not nan
complete_column_check = [df[key].notna() for key in template_keys]
# create a mask of complete rows to drop incomplete rows
complete_mask = (sum(complete_column_check) / len(template_keys)) >= 1
# only keep rows that have non-nan values for each required element of the template
df = df[complete_mask]


#############################################################
# WRITE INPUT FILES FROM TEMPLATE AND DATA
#############################################################

# make a directory to store the input files
os.makedirs(".\\input_files\\CX3_radical", exist_ok=True)

# loop through dataframe and write an input file for each line
for _, row in df.iterrows():

    # use molecule name to define filename with relative path
    filename = ".\\input_files\\CX3_radical\\{molecule}.inp".format(**row)

    print(f"Now writing {filename}")
    # generate input files
    with open(filename, "w") as f:
        f.writelines(CX3_radical_template.format(**row))

### (3.5) Running the Quantum Mechanics Models  
  
Now that students have properly generated input files, they will need to run their jobs on a computing cluster with the ORCA quantum mechanics modeling software installed. The intricacies or running these calculations are out of scope for the current submission, and as such we will assume that the user can effectivly run the properly formatted input files.  

### (4) Parse Output Files to Extract Data  
  
Assuming that the jobs completed successfully, students should now have a plethora of data to sort through!This can be especially overwhelming for undergraduates new to physical chemistry and new to programming in general.  As an example of one simple analysis, we demonstrate output parsing with the essential computational chemistry library cclib, storing the parsed data in a more easily readable excel file.  

In [ ]:
#############################################################
# LOAD OUTPUT FILES AND SCRAPE DATA
#############################################################

# define relative path to output files
out_path = ".\\output_files\\CX3_radical"
# loop through directory and load appropriate output files
parsed_rows = []
for folder in os.listdir(out_path):
    folder_path = "\\".join([out_path, folder])

    # use the cclib library to read the output file
    out_file = "\\".join([folder_path, folder + ".out"])
    ccout = cc.ccread(out_file)
    # check the documentation to see what other data you might want that these libraries will automatically supply
    # you can also choose to write your own parser, or find the data manually (especially for the property.txt files)


    # grab the data you want
    # you may choose to compute bond angles and other values here, rather than in excel
    atom_symbols = {"6" : "C", "1" : "H", "17" : "Cl", "9" : "F"}
    _, atomA, atomB, atomC = [atom_symbols[str(num)] for num in ccout.atomnos]
    Cx_out, Cy_out, Cz_out = ccout.atomcoords[-1][0]
    xA_out, yA_out, zA_out = ccout.atomcoords[-1][1]
    xB_out, yB_out, zB_out = ccout.atomcoords[-1][2]
    xC_out, yC_out, zC_out = ccout.atomcoords[-1][3]

    # append parsed row to list
    data_dict = {"atomA" : atomA, "atomB": atomB, "atomC" : atomC, 
                 "Cx_out" : Cx_out, "Cy_out" : Cy_out, "Cz_out" : Cz_out,
                 "xA_out" : xA_out, "yA_out" : yA_out, "zA_out" : zA_out,
                 "xB_out" : xB_out, "yB_out" : yB_out, "zB_out" : zB_out,
                 "xC_out" : xC_out, "yC_out" : yC_out, "zC_out" : zC_out,}
    parsed_row = pd.Series(data = data_dict)
    parsed_rows.append(parsed_row)

# convert parsed rows to dataframe
parsed_rows.reverse()
parsed_data = pd.DataFrame(parsed_rows)


#############################################################
# WRITE DATA TO DATA FRAME AND EXCEL
#############################################################

# reload original input dataframe (or you could make a completely new one)
CX3_dataframe_input = ".\\dataframes\\CX3_radical_dataframe_input.xlsx"
CX3_dataframe_output = ".\\dataframes\\CX3_radical_dataframe_output.xlsx"
# read file into pandas DataFrame object
df = pd.read_excel(CX3_dataframe_input)
df = df.sort_values(["atomA","atomB","atomC"], ignore_index= True)

# merge the parsed dataframe with the input dataframe
df.update(parsed_data)
# drop the duplicate columns

# update the dataframe excel file with new data
# note this will over-write the current file, so any formatting or plots will be lost
# you could save to a new name to avoid this issue
df.to_excel(CX3_dataframe_output, index=False)